- Original kaggle notebook: https://www.kaggle.com/danielwolffram/cord-19-create-dataframe

- 原本取的檔名: edited_outer_on_pdf

Some exploration and preprocessing to create one dataframe and export it as .csv.

Adjusted to data from 2020-05-01.

# Load Packages

In [43]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100)

from tqdm import tqdm
tqdm.pandas()

import regex as re
import glob
import json

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load and Prepare Data

To read the JSON files we follow [COVID EDA: Initial Exploration Tool](https://www.kaggle.com/ivanegapratama/covid-eda-initial-exploration-tool).

In [2]:
root_path = 'raw/'
metadata_path = f'{root_path}metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
        'pubmed_id': str,
        'Microsoft Academic Paper ID': str, 
        'doi': str
    }, 
    parse_dates=['publish_time'])
meta_df = meta_df.rename(columns = {'source_x': 'source'})

display(meta_df.isnull().sum())
print(len(meta_df) - meta_df.has_pdf_parse.sum())
meta_df.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


cord_uid                           0
sha                            14124
source                             0
title                            163
doi                             4086
pmcid                          11748
pubmed_id                      18559
license                            0
abstract                       11130
publish_time                       8
authors                         2591
journal                         7059
Microsoft Academic Paper ID    58923
WHO #Covidence                 58119
arxiv_id                       59211
has_pdf_parse                      0
has_pmc_xml_parse                  0
full_text_file                  8461
url                              440
dtype: int64

14124


,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synt...,10.1093/emboj/20.24.7220,PMC125340,11742998,unk,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; Spaan, Willy J.M.; Snijder, Eric J.",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=render
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a coronavirus receptor in the CEA family",10.1093/emboj/21.9.2076,PMC125375,11980704,unk,CEACAM1 is a member of the carcinoembryonic antigen (CEA) family. Isoforms of murine CEACAM1 ser...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu, Jin-huan; Bergelson, Jeffrey M.; Duke, Norma; Zh...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=render


In [3]:
# # test
# def highlight(col):
#     '''
#     highlight the maximum in a Series yellow.
#     '''
    
#     indicator = color_indicator.loc[duplicated['cord_uid'], col.name]
#     return ['background-color: yellow' if val else '' for val in indicator]

# duplicated = meta_df[meta_df.duplicated('cord_uid', keep=False)]
# color_indicator = duplicated.groupby('cord_uid').nunique(dropna=False) > 1

# duplicated.style.apply(highlight)

In [4]:
meta_df['cord_uid'].nunique(), \
meta_df['sha'].nunique(), \
meta_df['title'].nunique()

(59851, 45748, 58676)

In [5]:
all_json = glob.glob(f'{root_path}/**/pdf_json/*.json', recursive=True)
all_json_pmc = glob.glob(f'{root_path}/**/pmc_json/*.json', recursive=True)

len(all_json), len(all_json_pmc)

(48410, 22851)

# pdf_json

In [6]:
# test
all_possibilities = []
for path in all_json[:100]:
    with open(path) as f:
        j = json.load(f)
    for entry in j['body_text']:
        all_possibilities.append(entry['section'])
all_possibilities = list(set(all_possibilities))

print(all_possibilities[29])
print(re.sub('[^A-Za-z]', '', all_possibilities[29]).lower())

C52
c


In [7]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            self.methods = []
            self.results = []

            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
                
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
                
            # methods
# TODO: 這裡的對法怪怪的，可以再改過，現在就是先全部擠成一坨然後比對
            methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']
            methods = [re.sub('[^A-Za-z]', '', type_).lower() for type_ in methods]
            
            for entry in content['body_text']:
                # remove numbers and spaces
                # 'Fig. S12' -> 'figs'
                section_title = re.sub('[^A-Za-z]', '', entry['section']).lower()
                
                if any(t in section_title for t in methods) : 
                    self.methods.append(entry['text'])
                    
            # Results
            results_synonyms = ['result']
            for entry in content['body_text']:
                section_title = re.sub('[^A-Za-z]', '', entry['section']).lower()
                if any(r in section_title for r in results_synonyms) :
                    self.results.append(entry['text'])
            
            # seperate paragraphs
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            self.methods = '\n'.join(self.methods)
            self.results = '\n'.join(self.results)

    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'

# test
first_row = FileReader(all_json[0])
print(first_row)

bf23011551b21af6538e43974e74e6078f538028: ... Human infection by avian influenza virus (AIV) subtypes H7 1,2 and H9N2 3 has been reported in various countries over the past few years. Recent research reveals that AIV subtype H7 shares some proper...


In [8]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'methods': [], 'results': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
        
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['methods'].append(content.methods)
    dict_['results'].append(content.results)

papers = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'methods', 'results'])
papers.head()

Processing index: 0 of 48410
Processing index: 4841 of 48410
Processing index: 9682 of 48410
Processing index: 14523 of 48410
Processing index: 19364 of 48410
Processing index: 24205 of 48410
Processing index: 29046 of 48410
Processing index: 33887 of 48410
Processing index: 38728 of 48410
Processing index: 43569 of 48410


,paper_id,abstract,body_text,methods,results
0,bf23011551b21af6538e43974e74e6078f538028,,"Human infection by avian influenza virus (AIV) subtypes H7 1,2 and H9N2 3 has been reported in v...",,
1,c07b523c9bf9af72f5b6ee8b586b13ef619bb3ca,Recent advances in the development and discovery of pharmacological interventions within the ubi...,The ubiquitin-proteasome system (UPS) controls the turnover and biological function of most prot...,,
2,04908565dc76425cc6a3fabc1dff0bce3860e3c2,The history of agriculture includes many animal and plant disease events that have had major con...,"An emerging risk to plant, animal and/or human health may be defined as a risk resulting from a ...",,
3,c28ae565308bfabbd784a23fa065a59a49d8cfc3,,"Morbilliviruses belong to the order Mononegavirales, family Paramyxoviridae, and include measles...",,
4,82e80640af0a98cc07c911c9476f693c5e9a62cf,Beijing has been one of the epicenters attacked most severely by the SARS-CoV (severe acute resp...,"Accumulated number of probable cases and deaths of SARS in Beijing, China, and the world. Data s...",,


In [9]:
# test
(papers.methods.str.len() == 0).sum(), (papers.results.str.len() == 0).sum()

(34693, 37588)

In [10]:
# 找出 methods 或 results 的段落標題有符合的 paper
indicator = (papers.results.str.len() != 0) | (papers.methods.str.len() != 0)
papers[indicator].shape

(18591, 5)

In [11]:
# df = pd.merge(meta_df, papers, left_on='sha', right_on='paper_id', how='right', suffixes=['', 'pdf']).drop('sha', axis=1)

# print(df.columns)
# print(df.shape)
# df.head()

In [12]:
# TODO: 也許可以之後再做
df_outer = pd.merge(meta_df, papers, left_on='sha', right_on='paper_id', how='outer', suffixes=['', '_pdf']).drop('sha', axis=1)

print(df_outer.columns)
print(df_outer.shape)
df_outer.head()

Index(['cord_uid', 'source', 'title', 'doi', 'pmcid', 'pubmed_id', 'license',
       'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'arxiv_id',
       'has_pdf_parse', 'has_pmc_xml_parse', 'full_text_file', 'url',
       'paper_id', 'abstract_pdf', 'body_text', 'methods', 'results'],
      dtype='object')
(64967, 23)


,cord_uid,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,...,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,paper_id,abstract_pdf,body_text,methods,results
0,zjufx4fo,PMC,Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synt...,10.1093/emboj/20.24.7220,PMC125340,11742998,unk,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; Spaan, Willy J.M.; Snijder, Eric J.",...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=render,b2897e1277f56641193a6db73825f707eed3e4c9,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5¢ end of the genome fused...,The genetic information of RNA viruses is organized very ef®ciently. Practically every nucleotid...,"Site-directed mutagenesis, RNA transfections and immuno¯uorescence analysis Site-directed mutage...",EAV genome replication is not signi®cantly affected by leader TRS and body TRS mutations To diss...
1,ymceytj3,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a coronavirus receptor in the CEA family",10.1093/emboj/21.9.2076,PMC125375,11980704,unk,CEACAM1 is a member of the carcinoembryonic antigen (CEA) family. Isoforms of murine CEACAM1 ser...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu, Jin-huan; Bergelson, Jeffrey M.; Duke, Norma; Zh...",...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=render,e3d0d482ebd9a8ba81c254cc433f314142e72174,contributed equally to this work CEACAM1 is a member of the carcinoembryonic antigen (CEA) famil...,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...,,"Molecular structure of msCEACAM1a [1, 4] The msCEACAM1a[1,4] protein analyzed in this paper cont..."
2,wzj2glte,PMC,Synthesis of a novel hepatitis C virus protein by ribosomal frameshift,10.1093/emboj/20.14.3840,PMC125543,11447125,no-cc,Hepatitis C virus (HCV) is an important human pathogen that affects ∼100 million people worldwid...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; Lu, Wen; Strohecker, Anne; Govindarajan, Sugantha;...",...,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC125543/,00b1d99e70f779eb4ede50059db469c65e8c1469,,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of~9.6 kb. Infection...,,Synthesis of a 17 kDa protein from a coding sequence that overlaps the core protein reading fram...
3,2sfqsfm1,PMC,Structure of coronavirus main proteinase reveals combination of a chymotrypsin fold with an extr...,10.1093/emboj/cdf327,PMC126080,12093723,unk,"The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M(pro), a pro...",2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, Jeroen R.; Siddell, Stuart G.; Ziebuhr, John; Hilge...",...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc126080?pdf=render,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,"The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M pro , a pro...","Transmissible gastroenteritis virus (TGEV) belongs to the Coronaviridae, a family of positive-st...",Protein puri®cation and crystallization Recombinant TGEV M pro was expressed and puri®ed as prev...,Structure determination by MAD phasing The presence of 10 methionine residues in the TGEV M pro ...
4,i0zym7iq,PMC,Discontinuous and non-discontinuous subgenomic RNA transcription in a nidovirus,10.1093/emboj/cdf635,PMC136939,12456663,unk,"Arteri-, corona-, toro- and roniviruses are evolutionarily related positive-strand RNA viruses, ...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M.; de Groot, R.J.",...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc136939?pdf=render,dde02f11923815e6a16a31dd6298c46b109c5dfa,"corona-, toro-and roniviruses are evolutionarily related positive-strand RNA viruses, united in ...",Positi

# pmc_json

This only contains the full text - no abstracts!

In [13]:
# TODO: 把 pdf, pmc 的 FileReader 整合成一個 class
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.body_text = []
            self.methods = []
            self.results = []

            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            # Methods
            methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha()) #remove numbers and spaces
                if any(m in section_title for m in [''.join(x.lower() for x in m if x.isalpha()) for m in methods]) : 
                    self.methods.append(entry['text'])
            # Results
            results_synonyms = ['result']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha())
                if any(r in section_title for r in results_synonyms) :
                    self.results.append(entry['text'])
                    
            self.body_text = '\n'.join(self.body_text)
            self.methods = '\n'.join(self.methods)
            self.results = '\n'.join(self.results)

    def __repr__(self):
        return f'{self.paper_id}: {self.body_text[:200]}...'
first_row = FileReader(all_json_pmc[0])
print(first_row)

PMC6810424: ...


In [14]:
dict_ = {'paper_id': [], 'body_text': [], 'methods': [], 'results': []}
for idx, entry in enumerate(all_json_pmc):
    if idx % (len(all_json_pmc) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json_pmc)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    dict_['methods'].append(content.methods)
    dict_['results'].append(content.results)

pmc_text = pd.DataFrame(dict_, columns=['paper_id', 'body_text', 'methods', 'results'])

print(pmc_text.shape)
pmc_text.head()

Processing index: 0 of 22851
Processing index: 2285 of 22851
Processing index: 4570 of 22851
Processing index: 6855 of 22851
Processing index: 9140 of 22851
Processing index: 11425 of 22851
Processing index: 13710 of 22851
Processing index: 15995 of 22851
Processing index: 18280 of 22851
Processing index: 20565 of 22851
Processing index: 22850 of 22851
(22851, 4)


,paper_id,body_text,methods,results
0,PMC6810424,,,
1,PMC2289574,,,
2,PMC4916766,Successful intracellular pathogens often use strategies to gain access to cellular compartments ...,"For isolation of peripheral blood mononuclear cells (PBMCs), we obtained unidentified, discarded...","To determine whether MTb infection induces transcriptional activation of the IFITMs, we infected..."
3,PMC5857709,"The last decade has witnessed the emergence of several emerging infectious diseases, including t...",This survey was fielded in Singapore between November 2012 and February 2013 by a market researc...,\nTable 3 presents demographic characteristics for the 500 respondents and for a comparable popu...
4,PMC7031771,The on-going Coronavirus disease (COVID-19) outbreak in China has become the world's leading hea...,,


In [15]:
# Careful, some of the new texts are empty strings!
display(pmc_text[pmc_text.body_text == ''].head())
print(pmc_text.shape)

print('after dropping empty body_texts...')

pmc_text = pmc_text[pmc_text.body_text != '']
display(pmc_text[pmc_text.body_text == ''].head())
print(pmc_text.shape)

,paper_id,body_text,methods,results
0,PMC6810424,,,
1,PMC2289574,,,
9,PMC6252950,,,
12,PMC2119224,,,
23,PMC6735769,,,


(22851, 4)
after dropping empty body_texts...


,paper_id,body_text,methods,results


(20440, 4)


In [16]:
# merge meta_df, pdfs and pmcs

# TODO: 原本是 outer 在 pdf, 然後 pmc，如果都 outer 在 meta 上，後面會有問題
df = pd.merge(meta_df, papers, left_on='sha', right_on='paper_id', how='right', suffixes=['_meta', '_pdf'])
df = pd.merge(df, pmc_text, left_on='pmcid', right_on='paper_id', how='left', suffixes=['_pdf', '_pmc'])

# 丟掉被用來 merge 的 columns
# df = df.drop(['pmcid', 'paper_id_pdf', 'paper_id_pmc'], axis=1)
# 丟掉其他似乎沒用的 columns
# df = df.drop(['has_pdf_parse', 'has_pmc_xml_parse', 'Microsoft Academic Paper ID', 'WHO #Covidence']

print(df.shape)
print(df.columns)
df.head()

(48425, 28)
Index(['cord_uid', 'sha', 'source', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract_meta', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'arxiv_id',
       'has_pdf_parse', 'has_pmc_xml_parse', 'full_text_file', 'url',
       'paper_id_pdf', 'abstract_pdf', 'body_text_pdf', 'methods_pdf',
       'results_pdf', 'paper_id_pmc', 'body_text_pmc', 'methods_pmc',
       'results_pmc'],
      dtype='object')


,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract_meta,publish_time,...,url,paper_id_pdf,abstract_pdf,body_text_pdf,methods_pdf,results_pdf,paper_id_pmc,body_text_pmc,methods_pmc,results_pmc
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synt...,10.1093/emboj/20.24.7220,PMC125340,11742998,unk,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused...,2001-12-17,...,http://europepmc.org/articles/pmc125340?pdf=render,b2897e1277f56641193a6db73825f707eed3e4c9,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5¢ end of the genome fused...,The genetic information of RNA viruses is organized very ef®ciently. Practically every nucleotid...,"Site-directed mutagenesis, RNA transfections and immuno¯uorescence analysis Site-directed mutage...",EAV genome replication is not signi®cantly affected by leader TRS and body TRS mutations To diss...,PMC125340,The genetic information of RNA viruses is organized very efficiently. Practically every nucleoti...,Site-directed mutagenesis of EAV leader and body TRSs was carried out as described by van Marle ...,"To dissect EAV RNA synthesis, we routinely use a full-length cDNA clone (van Dinten et al., 1997..."
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a coronavirus receptor in the CEA family",10.1093/emboj/21.9.2076,PMC125375,11980704,unk,CEACAM1 is a member of the carcinoembryonic antigen (CEA) family. Isoforms of murine CEACAM1 ser...,2002-05-01,...,http://europepmc.org/articles/pmc125375?pdf=render,e3d0d482ebd9a8ba81c254cc433f314142e72174,contributed equally to this work CEACAM1 is a member of the carcinoembryonic antigen (CEA) famil...,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...,,"Molecular structure of msCEACAM1a [1, 4] The msCEACAM1a[1,4] protein analyzed in this paper cont...",PMC125375,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...,"Nucleotide sequences encoding the first 236 amino acids of murine CEACAM1a[1,4], including the n...","The msCEACAM1a[1,4] protein analyzed in this paper contains the 202 extracellular amino acids of..."
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein by ribosomal frameshift,10.1093/emboj/20.14.3840,PMC125543,11447125,no-cc,Hepatitis C virus (HCV) is an important human pathogen that affects ∼100 million people worldwid...,2001-07-16,...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC125543/,00b1d99e70f779eb4ede50059db469c65e8c1469,,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of~9.6 kb. Infection...,,Synthesis of a 17 kDa protein from a coding sequence that overlaps the core protein reading fram...,PMC125543,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of ∼9.6 kb. Infectio...,"For the construction of pCMV-CC, the HCV-1 core protein-coding sequence was isolated by PCR usin...",An inspection of the HCV-1 core protein-coding sequence revealed an overlapping coding sequence ...
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase reveals combination of a chymotrypsin fold with an extr...,10.1093/emboj/cdf327,PMC126080,12093723,unk,"The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M(pro), a pro...",2002-07-01,...,http://europepmc.org/articles/pmc126080?pdf=render,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,"The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M pro , a pro...","Transmissible gastroenteritis virus (TGEV) belongs to the Coronaviridae, a family of positive-st...",Protein puri®cation and crystallization Recombinant TGEV M pro was expressed and puri®ed as prev...,Structure determination by MAD phasing The 

In [17]:
# has_pdf_parse but no corresponding jsons
indicator = df['has_pdf_parse'] == True & df['body_text_pdf'].isna()
cols = ['has_pdf_parse', 'abstract_pdf', 'body_text_pdf', 'methods_pdf', 'results_pdf']
display(df[indicator][cols].head())
print(indicator.sum())

# has_pmc_parse but no corresponding jsons
indicator = df['has_pmc_xml_parse'] == True & df['body_text_pmc'].isna()
cols = ['has_pmc_xml_parse', 'body_text_pmc', 'methods_pmc', 'results_pmc']
display(df[indicator][cols].head())
print(indicator.sum())

,has_pdf_parse,abstract_pdf,body_text_pdf,methods_pdf,results_pdf


0


,has_pmc_xml_parse,body_text_pmc,methods_pmc,results_pmc
91,True,NaN,NaN,NaN
92,True,NaN,NaN,NaN
270,True,NaN,NaN,NaN
439,True,NaN,NaN,NaN
444,True,NaN,NaN,NaN


2178


# Exploration/Cleaning

### Different Abstract in Metadata and JSON files

In [18]:
different_abstracts = df[df.abstract_meta != df.abstract_pdf]

print(different_abstracts.shape)
with pd.option_context('display.max_colwidth', 100):
    display(different_abstracts[['abstract_meta', 'abstract_pdf', 'url']].tail(10))

(44678, 28)


,abstract_meta,abstract_pdf,url
48415,NaN,,NaN
48416,NaN,,NaN
48417,NaN,Glycyrrhetic acid polyglycosides were synthesized in one-pot via cationic ring-opening condensat...,NaN
48418,NaN,A purified recombinant spike (S) protein was studied for its effect on stimulating human periphe...,NaN
48419,NaN,Although susceptibility of neurons in the brain to microbial infection is a major determinant of...,NaN
48420,NaN,"A number of features were included in the final models, including article counts, indicator coun...",NaN
48421,NaN,,NaN
48422,NaN,Background-Since the turn of the century seven new respiratory viruses have infected man and two...,NaN
48423,NaN,"Genetic factors are presumed to influence the incidence of feline infectious peritonitis (FIP), ...",NaN
48424,NaN,,NaN


Checking some of the files online, it seems that where the abstract is missing in the metadata, the abstract in the JSON file is simply the beginning of the text.

下列以一文章為例。

## Changes of CT findings in a 2019 novel coronavirus (2019-nCoV) pneumonia patient
於 all_json 的第 20023 篇，  
url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7107315/pdf/hcaa038.pdf  
sha: 'b7555d21978c7989d2fe322f04f952f90b103d82'

abstracts 為下列，可以發現標題不為 "Abstracts":
### Learning points for clinicians
The gold standard for diagnosis of 2019 novel coronavirus (2019-nCoV) infection is the new coronavirus
nucleic acid in swabs, sputum, secretions from the lower
respiratory tract or blood. However, due to the quite low
sensitivity, short of supply and a relatively long period of
new coronavirus nucleic acid kits, suspected cases with
false-negative results remain a severe problem. Besides,
a long incubation period (3–7 days, up to 14 days) and superior transmission capacity of new viruses contribute to
the fast spread of 2019-nCoV. Under the circumstances,
computed tomography imaging is not only useful for
the detection, location of lesions but also helpful in
the evaluation of the dynamic changes of patients with
2019-nCoV.

In [19]:
# json 有 abstract, 但 meta 沒有, 而且有網址的 data
indicator = (df.abstract_meta.isnull()) & (df.abstract_pdf != '') & (~df.url.isnull())
different_abstracts.loc[
    indicator,
    ['abstract_meta', 'abstract_pdf', 'url', 'body_text_pdf', 'body_text_pmc']
].head()

,abstract_meta,abstract_pdf,url,body_text_pdf,body_text_pmc
23,NaN,58 ARDS = acute respiratory distress syndrome; HGMB1 = high mobility group B1; ICU = intensive c...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC420071/,John Marshall (University of Toronto) kicked off the meeting with an informative review of the e...,"The eighth annual Toronto Critical Care Symposium was held from 30 October to 1 November 2003, i..."
79,NaN,We agree with Doug Altman that correspondence is an essential part of medical publishing-acting ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1140948/,Editors of medical journals accept that published research should be open to comment and correct...,In their attempt to defend heterosexual transmission as the driving force for HIV epidemics in s...
103,NaN,"Diversity' is now a magic word, showing up everywhere and often used to embrace many aspects of ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1193560/,"'Diversity' is now a magic word, showing up everywhere and often used to embrace many aspects of...","‘Diversity’ is now a magic word, showing up everywhere and often used to embrace many aspects of..."
209,NaN,"York, UK, was a good choice for the venue of this workshop because the history of nucleolar rese...",http://europepmc.org/articles/pmc1559673?pdf=render,We now have working catalogues of the components of human and plant (Arabidopsis) nucleoli. A fe...,We now have working catalogues of the components of human and plant (Arabidopsis) nucleoli. A fe...
607,NaN,for fi eld collections; J. Andre for research permits; T. Kreeger for access to his facility; R....,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2600301/,"To the Editor: In the November 2007 issue of Emerging Infectious Diseases, Gayer et al. (1) desc...","In the November 2007 issue of Emerging Infectious Diseases, Gayer et al. (1) describe how confli..."


In [20]:
# TODO: 這邊數字怪怪的, 左上原本應為 0, 可能跟 dataframe merge 的方法有關
print('     null  empty')

# missing abstracts in json files
print('json', df['abstract_pdf'].isnull().sum(), (df['abstract_pdf'] =='').sum())

# missing abstracts in metadata
print('meta', df['abstract_meta'].isnull().sum(), (df['abstract_meta']=='').sum())

     null  empty
json 0 13737
meta 10993 0


Since the abstracts from the metadata seem more reliable we generally use these, but fill the missing values with the abstract from the extracted values from the JSON file.

可能因為 json 中的 abstract，它的標題並不一定為 abstract 的原因。

In [21]:
indicator = (df.abstract_meta.isnull()) & (df.abstract_pdf != '')
df.loc[indicator, 'abstract_meta'] = df.loc[indicator, 'abstract_pdf']

df['abstract_meta'].isnull().sum()

6515

In [22]:
# the remaining missing values are also empty in the json files
# TODO: 可能也因為 dataframe merge 的方法，這邊原本應該為 0

(df.abstract_meta.isnull() & (df.abstract_pdf!='')).sum()

0

In [23]:
df = df.rename(columns = {'abstract_meta': 'abstract'})
df = df.drop('abstract_pdf', axis=1)

print(df.columns)
print(df.shape)

Index(['cord_uid', 'sha', 'source', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'arxiv_id',
       'has_pdf_parse', 'has_pmc_xml_parse', 'full_text_file', 'url',
       'paper_id_pdf', 'body_text_pdf', 'methods_pdf', 'results_pdf',
       'paper_id_pmc', 'body_text_pmc', 'methods_pmc', 'results_pmc'],
      dtype='object')
(48425, 27)


We still have to compare the text body from pdf and pmc files.

# Quick comparison of both texts

In [24]:
print((df.body_text_pdf != df.body_text_pmc).sum(), df.shape)

48425 (48425, 27)


In [25]:
indicator = (df.body_text_pdf != df.body_text_pmc) & df.body_text_pmc.notnull()
display(df.loc[indicator, ['body_text_pdf', 'body_text_pmc']].head(10))

display(df.loc[2, 'body_text_pdf'][:500])
print()
display(df.loc[2, 'body_text_pmc'][:500])

,body_text_pdf,body_text_pmc
0,The genetic information of RNA viruses is organized very ef®ciently. Practically every nucleotid...,The genetic information of RNA viruses is organized very efficiently. Practically every nucleoti...
1,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...
2,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of~9.6 kb. Infection...,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of ∼9.6 kb. Infectio...
3,"Transmissible gastroenteritis virus (TGEV) belongs to the Coronaviridae, a family of positive-st...","Transmissible gastroenteritis virus (TGEV) belongs to the Coronaviridae, a family of positive-st..."
4,Positive (+)-strand RNA viruses have developed a wide range of strategies to express their genes...,Positive (+)-strand RNA viruses have developed a wide range of strategies to express their genes...
5,"Rhinoviruses have been associated with 40% to 65% of ""common colds"" studied by methods of virus ...","Rhinoviruses have been associated with 40% to 65% of ""common colds"" studied by methods of virus ..."
6,Charles Darwin recognized that the distribution and form of parasites was evolutionarily signifi...,Charles Darwin recognized that the distribution and form of parasites was evolutionarily signifi...
7,The elongation phase of protein synthesis is a precise process and mechanisms exist to promote t...,The elongation phase of protein synthesis is a precise process and mechanisms exist to promote t...
8,Salmonella enterica serovar Typhimurium (S.typhimurium) is a bacterial pathogen that invades and...,Salmonella enterica serovar Typhimurium (S.typhi murium) is a bacterial pathogen that invades an...
9,"patient group, a further significant increase of the allele was seen in the ""Severe cases"" patie...","In late April 2003, Taiwan was stroked by an epidemic of severe acute respiratory syndrome (SARS..."


'Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of~9.6 kb. Infection by this virus frequently leads to chronic infection, which in turn may lead to severe liver diseases including liver cirrhosis and hepatocellular carcinoma. The ®rst HCV genomic sequence, which was named the HCV-1 sequence, was isolated in 1989 (Choo et al., 1989) . Since then, many more HCV sequences have been reported. Based on their sequence similarities, these HCV sequences have been grouped into'

'Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of ∼9.6 kb. Infection by this virus frequently leads to chronic infection, which in turn may lead to severe liver diseases including liver cirrhosis and hepatocellular carcinoma. The first HCV genomic sequence, which was named the HCV-1 sequence, was isolated in 1989 (Choo et al., 1989). Since then, many more HCV sequences have been reported. Based on their sequence similarities, these HCV sequences have been grouped int'

可能擷取的段落不一樣，原因為 paper 格式不同，下面為另外一個例子，為原本 dataframe 的第 1242 筆 data:

url:'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3100511/'

### df.iloc[1242].body_text_pdf[:500]:  
'The global prevalence of the dengue virus (DENV) has grown dramatically in recent decades, and it is now endemic in .100 countries, with some 2.5 billion people at risk of infection. Dengue is an arthropod-borne flavivirus that can be subdivided into the 4 major serotypes (DEN-1-DEN-4). Most dengue infections either are asymptomatic or lead to a self-limiting febrile illness, dengue fever. In some cases the illness is more severe, leading to dengue hemorrhagic fever (DHF) with severe plasma leak'
    
### df.iloc[1242].body_text_pmc[:500]:  
'The dengue-2 strain, 16681, was grown in C6/36 cells, Vero cells, and monocyte-derived dendritic cells (DCs). Cell-free supernatants were used either neat or after concentration by ultracentrifugation at 45,000 rpm for 4 h at 4°C, and the virus pellet was resuspended in 1.5% fetal bovine serum (FBS)/Leibovitz L-15. To concentrate large volumes of low-titer DENV supernatant, DENV were precipitated with 10% Polyethylene glycol 8000 (Sigma) before ultracentrifugation. U937 were maintained in 10% FB'

Where available we use the text from the pmc file (body_text_pmc), trusting the statement that it is of higher quality.

In [26]:
# TODO: 跟 abstract 一樣怪怪的, 左上原本應為 0, 可能跟 dataframe merge 的方法有關
print('      pdf   pmc')

print('null ', df.body_text_pdf.isnull().sum(), df.body_text_pmc.isnull().sum())

print('empty', (df.body_text_pdf == '').sum(), (df.body_text_pmc == '').sum())

      pdf   pmc
null  0 30109
empty 0 0


In [27]:
indicator = df.body_text_pmc.notnull()
df.loc[indicator, 'body_text_pdf'] = df.loc[indicator, 'body_text_pmc']

In [28]:
# TODO: 可能也因為 dataframe merge 的方法，這邊原本應該為 0
df['body_text_pdf'].isnull().sum()

0

In [29]:
df = df.rename(columns = {'body_text_pdf': 'body_text'})
df = df.drop('body_text_pmc', axis=1)

print(df.columns)

Index(['cord_uid', 'sha', 'source', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'arxiv_id',
       'has_pdf_parse', 'has_pmc_xml_parse', 'full_text_file', 'url',
       'paper_id_pdf', 'body_text', 'methods_pdf', 'results_pdf',
       'paper_id_pmc', 'methods_pmc', 'results_pmc'],
      dtype='object')


In [30]:
df.loc[df['methods_pmc'].notnull(), ['methods_pdf', 'methods_pmc', 'url']].head()

,methods_pdf,methods_pmc,url
0,"Site-directed mutagenesis, RNA transfections and immuno¯uorescence analysis Site-directed mutage...",Site-directed mutagenesis of EAV leader and body TRSs was carried out as described by van Marle ...,http://europepmc.org/articles/pmc125340?pdf=render
1,,"Nucleotide sequences encoding the first 236 amino acids of murine CEACAM1a[1,4], including the n...",http://europepmc.org/articles/pmc125375?pdf=render
2,,"For the construction of pCMV-CC, the HCV-1 core protein-coding sequence was isolated by PCR usin...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC125543/
3,Protein puri®cation and crystallization Recombinant TGEV M pro was expressed and puri®ed as prev...,All six copies (designated A–F) of the TGEV Mpro in the asymmetric unit of the crystal could be ...,http://europepmc.org/articles/pmc126080?pdf=render
4,Cells and virus BEV was grown in equine dermis (Ederm) cells (American Type Tissue Culture Colle...,BEV was grown in equine dermis (Ederm) cells (American Type Tissue Culture Collection). A BEV st...,http://europepmc.org/articles/pmc136939?pdf=render


In [31]:
# TODO: 這邊數字怪怪的, 右上原本應為 0, 可能跟 dataframe merge 的方法有關
print('    null  empty')
print('pdf', (df['methods_pdf'] == '').sum(), df['methods_pdf'].isnull().sum())
print('pmc', (df['methods_pmc'] == '').sum(), df['methods_pmc'].isnull().sum())

    null  empty
pdf 34708 0
pmc 6622 30109


In [32]:
# use methods_pmc when it's available
indicator = (df.methods_pmc.notnull()) & (df.methods_pmc != '')
df.loc[indicator, 'methods_pdf'] = df.loc[indicator, 'methods_pmc']

# same for results
indicator = (df.results_pmc.notnull()) & (df.results_pmc != '')
df.loc[indicator, 'results_pdf'] = df.loc[indicator, 'results_pmc']

# TODO: 這邊數字怪怪的, 右上原本應為 0, 可能跟 dataframe merge 的方法有關
print('    null  empty')
print('pdf', (df['methods_pdf'] == '').sum(), df['methods_pdf'].isnull().sum())
print('pmc', (df['methods_pmc'] == '').sum(), df['methods_pmc'].isnull().sum())

df.rename(columns = {'methods_pdf': 'methods', 'results_pdf': 'results'}, inplace=True)
df.drop(columns=['methods_pmc', 'results_pmc'], inplace=True)

print(df.columns)
df.head()

    null  empty
pdf 28791 0
pmc 6622 30109
Index(['cord_uid', 'sha', 'source', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'arxiv_id',
       'has_pdf_parse', 'has_pmc_xml_parse', 'full_text_file', 'url',
       'paper_id_pdf', 'body_text', 'methods', 'results', 'paper_id_pmc'],
      dtype='object')


,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,paper_id_pdf,body_text,methods,results,paper_id_pmc
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synt...,10.1093/emboj/20.24.7220,PMC125340,11742998,unk,Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused...,2001-12-17,...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=render,b2897e1277f56641193a6db73825f707eed3e4c9,The genetic information of RNA viruses is organized very efficiently. Practically every nucleoti...,Site-directed mutagenesis of EAV leader and body TRSs was carried out as described by van Marle ...,"To dissect EAV RNA synthesis, we routinely use a full-length cDNA clone (van Dinten et al., 1997...",PMC125340
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a coronavirus receptor in the CEA family",10.1093/emboj/21.9.2076,PMC125375,11980704,unk,CEACAM1 is a member of the carcinoembryonic antigen (CEA) family. Isoforms of murine CEACAM1 ser...,2002-05-01,...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=render,e3d0d482ebd9a8ba81c254cc433f314142e72174,Carcinoembryonic antigen (CEA; CD66e) was initially discovered as a tumor antigen (Gold and Free...,"Nucleotide sequences encoding the first 236 amino acids of murine CEACAM1a[1,4], including the n...","The msCEACAM1a[1,4] protein analyzed in this paper contains the 202 extracellular amino acids of...",PMC125375
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein by ribosomal frameshift,10.1093/emboj/20.14.3840,PMC125543,11447125,no-cc,Hepatitis C virus (HCV) is an important human pathogen that affects ∼100 million people worldwid...,2001-07-16,...,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC125543/,00b1d99e70f779eb4ede50059db469c65e8c1469,Hepatitis C virus (HCV) is a positive-stranded RNA virus with a genome size of ∼9.6 kb. Infectio...,"For the construction of pCMV-CC, the HCV-1 core protein-coding sequence was isolated by PCR usin...",An inspection of the HCV-1 core protein-coding sequence revealed an overlapping coding sequence ...,PMC125543
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase reveals combination of a chymotrypsin fold with an extr...,10.1093/emboj/cdf327,PMC126080,12093723,unk,"The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M(pro), a pro...",2002-07-01,...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc126080?pdf=render,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,"Transmissible gastroenteritis virus (TGEV) belongs to the Coronaviridae, a family of positive-st...",All six copies (designated A–F) of the TGEV Mpro in the asymmetric unit of the crystal could be ...,The presence of 10 methionine residues in the TGEV Mpro molecule suggested that selenomethionine...,PMC126080
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,PMC,Discontinuous and non-discontinuous subgenomic RNA transcription in a nidovirus,10.1093/emboj/cdf635,PMC136939,12456663,unk,"Arteri-, corona-, toro- and roniviruses are evolutionarily related positive-strand RNA viruses, ...",2002-12-01,...,NaN,True,True,custom_license,http://europepmc.org/articles/pmc136939?pdf=render,dde02f11923815e6a16a31dd6298c46b109c5dfa,Positive (+)-strand RNA viruses have developed a wide range of strategies to express their genes...,BEV was grown in equine dermis (Ederm) cells (American Type Tissue Culture Collection). A BEV st...,Although most RNA viruses provide their (+)strand transcripts with a 5′ cap (Furuichi and Shatki...,PMC136939


# Duplicates

In [33]:
# Some paper ids are duplicated
df['sha'].nunique(), df.shape

(43330, (48425, 24))

In [34]:
# TODO: sha 裡面會有 Nan
duplicated = df.duplicated('sha', keep=False) & df['sha'].notna()
print(duplicated.sum())
df.loc[duplicated, ['sha', 'body_text']].sort_values('sha').head()

28


,sha,body_text
31867,0ed3c6a5559cd73307184f51fc53ccc76da559bc,"In this Journal, Zhu et al. recently reported the results of their genomic analysis of multidrug..."
31866,0ed3c6a5559cd73307184f51fc53ccc76da559bc,"In this Journal, Zhu et al. recently reported the results of their genomic analysis of multidrug..."
31865,0ed3c6a5559cd73307184f51fc53ccc76da559bc,"In this Journal, Zhu et al. recently reported the results of their genomic analysis of multidrug..."
22633,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,A digestible vaccine sandwich VACCINE BIOTECHNOLOGY edited by James L. Bittle and Frederick L. M...
22634,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,A digestible vaccine sandwich VACCINE BIOTECHNOLOGY edited by James L. Bittle and Frederick L. M...


But luckily they also have the same text body. So we will just keep one article per paper_id.
Check for example [https://www.sciencedirect.com/science/article/pii/S1386653209701295?via%3Dihub](https://www.sciencedirect.com/science/article/pii/S1386653209701295?via%3Dihub) and [https://www.sciencedirect.com/science/article/pii/S1386653209701325?via%3Dihub](https://www.sciencedirect.com/science/article/pii/S1386653209701325?via%3Dihub) - they have the same content.

In [35]:
df = df.drop_duplicates(['sha', 'body_text'])    # sha, body_text 都一樣才會 drop
print(df.shape)
df[df.duplicated(['sha'], keep=False)].head(2)

(47884, 24)


,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,paper_id_pdf,body_text,methods,results,paper_id_pmc
14210,4fbr8fx8,bd3945ec343cc4535650c9425aa143f71716de7c,PMC,"The clinical importance of the nomenclature, evolution and taxonomy of human papillomaviruses",10.1016/j.jcv.2004.10.021,PMC7108213,15571999,no-cc,Human papillomaviruses (HPVs) are formally described by isolation of their circular double-stran...,2005-02-08,...,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7108213/,bd3945ec343cc4535650c9425aa143f71716de7c,"Until the late 1970s, papillomaviruses attracted little interest as they were only known as the ...",,,PMC7108213
14211,4fbr8fx8,bd3945ec343cc4535650c9425aa143f71716de7c,Elsevier,"The clinical importance of the nomenclature, evolution and taxonomy of human papillomaviruses",10.1016/j.jcv.2004.10.021,NaN,15753006,els-covid,Abstract Human papillomaviruses (HPVs) are formally described by isolation of their circular dou...,2005-03-31,...,NaN,True,False,custom_license,https://doi.org/10.1016/j.jcv.2004.10.021,bd3945ec343cc4535650c9425aa143f71716de7c,"Until the late 1970s, papillomaviruses attracted little interest as they were only known as the ...",,,NaN


In [36]:
# TODO: sha 裡面會有 Nan
# TODO: 這裡並沒有特別處理 sha 一樣，但 body_text 不一樣的 data，直接 drop
duplicated = df.duplicated('sha', keep=False) & df['sha'].notna()
df[duplicated].sort_values('sha').head(2)

,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,paper_id_pdf,body_text,methods,results,paper_id_pmc
22558,fjmchbew,58be092086c74c58e9067121a6ba4836468e7ec3,CZI,The Author's Response: Case of the Index Patient Who Caused Tertiary Transmission of Coronavirus...,10.3346/jkms.2020.35.e89,NaN,32080993,cc-by-nc,NaN,2020-01-01,...,NaN,True,False,noncomm_use_subset,https://doi.org/10.3346/jkms.2020.35.e89,58be092086c74c58e9067121a6ba4836468e7ec3,"administration reduces viral load. Also, the authors explain that LPV/r administration has also ...",,,NaN
22559,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patient Who Caused Tertiary Transmission of Coronavirus ...,10.3346/jkms.2020.35.e88,PMC7036343,32080992,cc-by-nc,NaN,2020-02-20,...,NaN,True,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7036343/,58be092086c74c58e9067121a6ba4836468e7ec3,I am grateful that Dr. Lim and his colleagues reported a case of COVID-19 that caused tertiary t...,,,PMC7036343


In [37]:
df = df.drop_duplicates(['sha'])
df['sha'].nunique(), df.shape

(43330, (43331, 24))

In [38]:
# TODO: 這個要處理，目前沒有做任何處理。
df[df.duplicated('pmcid', keep=False)]

,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,paper_id_pdf,body_text,methods,results,paper_id_pmc
21429,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical twins discordant for systemic lupus erythematosus,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, one of whom had systemic lupus erythematosus (SLE),...",1973-08-31,...,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9,aecbc613ebdab36753235197ffb4f35734b5ca63,"The patient (Fo, ) was a 58 year old mentally retarded white woman, born in a rural area of sout...",,,NaN
21430,b10e023s,55b8337a50d270af02549897d75e22bea89b2083,Elsevier,Epidemic viral gastroenteritis,10.1016/0002-9343(79)90457-1,NaN,222141,els-covid,Abstract Epidemic viral gastroenteritis is a significant world wide problem. In developed countr...,1979-06-30,...,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(79)90457-1,55b8337a50d270af02549897d75e22bea89b2083,"Epidemic viral gastroenteritis is a significant world wide problem. In developed countries, gast...",,,NaN
21431,i94lyfsh,ddd2ecf42ec86ad66072962081e1ce4594431f9c,Elsevier,Management of acute and chronic respiratory tract infections,10.1016/0002-9343(88)90456-1,NaN,3048091,els-covid,"Abstract Pharyngitis, bronchitis, and pneumonia represent the most common respiratory tract infe...",1988-09-16,...,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(88)90456-1,ddd2ecf42ec86ad66072962081e1ce4594431f9c,"Respiratory Tract Infections JERROLD J. ELLNER, M.D. Cleveland, CM Pharyngitis, bronchitis, and ...",,,NaN
21432,vs5yondw,a55cb4e724091ced46b5e55b982a14525eea1c7e,Elsevier,Acute bronchitis: Results of U.S. and European trials of antibiotic therapy,10.1016/0002-9343(92)90608-e,NaN,1621745,els-covid,"Abstract Acute bronchitis, an illness frequently encountered by primary-care physicians, is an i...",1992-06-22,...,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(92)90608-e,a55cb4e724091ced46b5e55b982a14525eea1c7e,"A cute bronchitis, an illness frequently encountered by primary-care physicians [1] , is an infl...",,"Four clinical trials (two in the United States, two in Europe) were initiated to study the relat...",NaN
21433,4sbuzcvn,60bf634cf15582a6b7f72eb3811ca5d8b8825c44,Elsevier,Determination of microsomal lauric acid hydroxylase activity by HPLC with flow-through radiochem...,10.1016/0003-2697(88)90093-0,NaN,3389520,els-covid,"Abstract An assay for the microsomal hydroxylation of lauric acid (LA), based on HPLC with flow-...",1988-04-30,...,NaN,True,False,custom_license,https://doi.org/10.1016/0003-2697(88)90093-0,60bf634cf15582a6b7f72eb3811ca5d8b8825c44,"Laurie acid (LA),' like prostaglandins (PGs) and leukotrienes, is hydroxylated at the w and (w-l...",,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42967,i2vg8zqc,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802,Elsevier,Clinical Study of Mesenchymal Stem Cell Treatment for Acute Respiratory Distress Syndrome Induce...,10.1016/j.eng.2020.02.006,NaN,NaN,els-covid,Abstract H7N9 viruses quickly spread between mammalian hosts and carry the risk of human-to-huma...,2020-02-28,...,NaN,True,False,custom_license,https://doi.org/10.1016/j.eng.2020.02.006,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802,Influenza A viruses (IAVs) are divided into multiple subtypes according to diversified viral sur...,,,NaN
43067,6b7o2cda,f864c42617b8702b776985577444149bbdd7b176,Elsevier,Clinical features of atypical 2019 novel coronavirus pneumonia with an initially negative RT-PCR...,10.1016/j.jinf.2020.02.008,NaN,32092387,els-covid,NaN,2020-02-22,...,NaN,True,False,custom_license,https://doi.org/10.1016/j.jinf.2020.02.008,f864c42617b8702b776985577444149bbdd7b176,". Chest CT imaging of the patient (a, c) Chest CT scans obtained at admission sh

In [39]:
# TODO: 需要再整理前處理過程，目前還沒清乾淨
df.isnull().sum()

cord_uid                           1
sha                                1
source                             1
title                             39
doi                             1065
pmcid                           8267
pubmed_id                      13280
license                            1
abstract                        5051
publish_time                       1
authors                          697
journal                         3720
Microsoft Academic Paper ID    42976
WHO #Covidence                 42808
arxiv_id                       42702
has_pdf_parse                      1
has_pmc_xml_parse                  1
full_text_file                     1
url                              121
paper_id_pdf                       0
body_text                          0
methods                            0
results                            0
paper_id_pmc                   25016
dtype: int64

# Some new columns for convenience

In [40]:
# some new columns for convenience
df['publish_year'] = df['publish_time'].dt.year#.fillna(-1)
# df['link'] = 'http://dx.doi.org/' + df.doi    # dataset now has url column

In [41]:
re.search('COVID', 'covid', re.IGNORECASE)

<regex.Match object; span=(0, 5), match='covid'>

In [44]:
def contain_covid19_keywords(row):
    
    covid19_keywords = ['COVID-19', 'covid', 'sar cov 2', 'SARS-CoV-2', '2019-nCov',
        '2019 ncov', 'SARS Coronavirus 2', '2019 Novel Coronavirus', 'coronavirus 2019', 'Wuhan coronavirus',
        'wuhan pneumonia', 'wuhan virus']
    return any(bool(re.search(kw, row['body_text'], re.IGNORECASE)) for kw in covid19_keywords)
    
df['is_covid19'] = df.progress_apply(contain_covid19_keywords, axis=1)
df['is_covid19'].sum()

100%|██████████| 43331/43331 [00:15<00:00, 2778.53it/s]


6367

# Language Detection to remove non-english articles and abstracts

In [45]:
from IPython.utils import io

with io.capture_output() as captured:
    !pip install scispacy
    !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
    !pip install spacy-langdetect
    !pip install spac scispacy spacy_langdetect https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.3/en_core_sci_lg-0.2.3.tar.gz

In [46]:
import scispacy
import spacy
import en_core_sci_lg
from spacy_langdetect import LanguageDetector

ModuleNotFoundError: No module named 'scispacy'

In [ ]:
# medium model
nlp = en_core_sci_lg.load(disable=["tagger", "ner"])
nlp.max_length = 2000000
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [ ]:
# simple example
doc = nlp('This is some English text. Das ist ein Haus. This is a house.')

print(doc._.language)
print()
for s in doc.sents:
    print(s._.language)

In [ ]:
# example for dataset
doc = nlp(df[df['sha'] == '1a8a4dbbaa94ced4ef6af69ec7a09d3fa4c0eece'].body_text.iloc[0])
doc[:500]

In [ ]:
doc_engl = ''
for s in doc.sents:
    if (s._.language['language'] == 'en'):
        doc_engl += s.text
        
doc_engl[:2000]

In [ ]:
df['body_text'].isna().sum()

In [ ]:
import langdetect
from tqdm import tqdm
tqdm.pandas()

In [ ]:
re.search('[A-Za-z]', df.loc[22574, 'body_text'])

In [ ]:
df[22560:22570]['body_text']

In [ ]:
def filter_and_detect_language(x):
    x = str(x[:2000]).strip()
    if re.search('[A-Za-z]', x):
        return langdetect.detect(x)
    else:
        return ''

# Check language of each text body (only use the first 2000 characters).
df['text_language'] = df['body_text'].progress_apply(filter_and_detect_language)
# 原本為 df['body_text'].progress_apply(lambda x: nlp(str(x[:2000]))._.language['language']) 慢很多

df.text_language.value_counts()

## Number of non-english texts to drop.

In [ ]:
print(df.loc[df.text_language != 'en'].shape)
df = df[df.text_language == 'en']
print(df.loc[df.text_language != 'en'].shape)

In [ ]:
# Check language of all abstracts

# df['abstract_lang'] = df.abstract.apply(lambda x: nlp(str(x))._.language['language'])

#  df[df.abstract.isnull()]

In [ ]:
# Number of non-english abstracts

# df[(df.abstract_lang != 'en') & (df.abstract.notnull())].abstract_lang.value_counts()

# Keep all english abstracts and those without abstract

# df = df[(df.abstract_lang == 'en') | (df.abstract.isnull())]

# df.shape

# df.paper_id.nunique()

# Analyze title/text body of the papers without abstract

# temp = df[df.abstract.isnull()].copy()

# def remove_non_english_sentences(doc):
#     doc = nlp(doc)
#     doc_engl = ''
#     for s in doc.sents:
#         if (s._.language['language'] == 'en'):
#             doc_engl += s.text 
#     return doc_engl

# remove_non_english_sentences(df[df.paper_id == '1a8a4dbbaa94ced4ef6af69ec7a09d3fa4c0eece'].body_text.iloc[0])

# temp['text_length'] = temp.body_text.apply(lambda x: len(x))

# temp['english_text'] = temp.body_text.apply(remove_non_english_sentences)

# temp['english_length'] = temp.english_text.apply(lambda x: len(x))

# temp.to_csv('df_english.csv', index=False)

# (temp.english_length/temp.text_length).hist()

# ((temp.english_length/temp.text_length)<0.8).sum()

# temp[((temp.english_length/temp.text_length)<0.8)].head()

# temp[temp.paper_id == '7925057cfe0cb75ae6079879cb2d22d23e42dfa5'].body_text.values[0][:500]

# temp[temp.paper_id == '617197cc751a9208cb0af1b4e31baeddc8d2e985'].body_text.values[0]

# temp[temp.paper_id == 'ca51b53fa512085e1aa166d5308602ff1666a90c'].body_text.values[0][:500]

# df = df.drop(temp[((temp.english_length/temp.text_length)<0.8)].index)

In [ ]:
# temp['title_lang'] = df.title.apply(lambda x: nlp(str(x))._.language['language'])

# temp.title_lang.value_counts()

# Too many false-positves. 

# temp[temp.paper_id == '6f6b7b1efffae7f3765f29fe801ab63dd35110bb'].body_text.values[0]

# temp[temp.title_lang == 'de']

# We check the beginning of each text body instead.

# temp['text_lang'] = df.body_text.apply(lambda x: nlp(str(x[:2000]))._.language['language'])

# temp.text_lang.value_counts()

# Number of non-english texts to drop.

# df.loc[temp[temp.text_lang != 'en'].index].shape

# df = df.drop(temp[temp.text_lang != 'en'].index)

# Extract Study Design/ Methodological Keywords

In [ ]:
# filter_dict = {
#     "discussion": ["conclusions","conclusion",'| discussion', "discussion",  'concluding remarks',
#                    'discussion and conclusions','conclusion:', 'discussion and conclusion',
#                    'conclusions:', 'outcomes', 'conclusions and perspectives', 
#                    'conclusions and future perspectives', 'conclusions and future directions'],
#     "results": ['executive summary', 'result', 'summary','results','results and discussion','results:',
#                 'comment',"findings"],
#     "introduction": ['introduction', 'background', 'i. introduction','supporting information','| introduction'],
#     "methods": ['methods','method','statistical methods','materials','materials and methods',
#                 'data collection','the study','study design','experimental design','objective',
#                 'objectives','procedures','data collection and analysis', 'methodology',
#                 'material and methods','the model','experimental procedures','main text',],
#     "statistics": ['data analysis','statistical analysis', 'analysis','statistical analyses', 
#                    'statistics','data','measures'],
#     "clinical": ['diagnosis', 'diagnostic features', "differential diagnoses", 'classical signs','prognosis', 'clinical signs', 'pathogenesis',
#                  'etiology','differential diagnosis','clinical features', 'case report', 'clinical findings',
#                  'clinical presentation'],
#     'treatment': ['treatment', 'interventions'],
#     "prevention": ['epidemiology','risk factors'],
#     "subjects": ['demographics','samples','subjects', 'study population','control','patients', 
#                'participants','patient characteristics'],
#     "animals": ['animals','animal models'],
#     "abstract": ["abstract", 'a b s t r a c t','author summary'], 
#     "review": ['review','literature review','keywords']}

In [ ]:
study_designs = {'RCT': ['RCT', 'randomized controlled trial', 'randomised controlled trial', 'randomized control trial', 'randomised control trial',
                         'randomized clinical trial','randomised clinical trial'], 
                'time series analysis': ['time series analysis', 'time series', 'survival analysis'],
                'retrospective cohort': ['retrospective cohort'],
                'cross-sectional case-control': ['cross-sectional case-control', 'cross sectional case control', 'cross-sectional case control'],
                'prospective case-control': ['prospective case-control', 'prospective case control'],
                'matched case-control': ['matched case-control', 'matched case control'],
                'medical records review': ['medical records review'],
                'prevalence survey': ['prevalence survey'],
                'syndromic surveillance': ['syndromic surveillance'],
                'systematic review': ['systematic review'],
                'meta-analysis': ['meta-analysis', 'meta analysis', 'meta-syntheses'],
                'interventional study': ['interventional study'],
                'association': ['association', 'associated with'],
                 'p-value': ['p-value', 'p value'],
                 'pseudo-randomized controlled trial': ['pseudo-randomized controlled trial', 'pseudo-randomised controlled trial']
                }

# Keywords from  [https://docs.google.com/spreadsheets/d/1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E/edit#gid=1217643351](https://docs.google.com/spreadsheets/d/1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E/edit#gid=1217643351)

generic_keywords = ['estimation',
 'prevalence survey',
 'response rate',
 'incidence',
 'psychometric evaluation of instrument',
 'median time to event',
 'pooled OR',
 'd-pooled',
 'randomized controlled trial',
 'non-randomized',
 'allocation method',
 'Cochrane review',
 'Cox proportional hazards',
 'gamma',
 'Weibull',
 'pseudo-randomised',
 'chart review',
 'log odds',
 'surveillance',
 'time-to-event analysis',
 'pooled adjusted odds ratio',
 'pooled relative risk',
 'data abstraction forms',
 'frequency',
 'etiology logistic regression',
 'exclusion criteria',
 'eligibility criteria',
 'right-censored',
 'pooled odds ratio',
 'non-comparative study',
 'medical records review',
 'CONSORT',
 'number of controls per case',
 'quasi-randomised',
 'risk of bias',
 'publication bias',
 'syndromic surveillance',
 'truncated',
 'longitudinal',
 'matching criteria',
 'double-blind',
 "Cohen's d",
 'registry data',
 'Adjusted Odds Ratio',
 'questionnaire development',
 'Kaplan-Meier',
 'heterogeneity',
 'recruitment',
 'randomization method',
 'censoring',
 'meta-analysis',
 'non-randomised',
 'β',
 'electronic medical records',
 'eligibility',
 'cross-sectional survey',
 'PRISMA',
 'prevalence',
 'inclusion criteria',
 'control arm',
 'protocol',
 'pooled risk ratio',
 'non-response bias',
 'baseline',
 'retrospective chart review',
 'survival analysis',
 'logistic regression',
 'blind',
 'exposure status',
 'randomized',
 'associated with',
 'lognormal',
 'systematic review',
 'RCT',
 'randomised',
 'survey instrument',
 'interrater reliability',
 'randomisation',
 'pooled RR',
 'hazard ratio',
 'AOR',
 'potential confounders',
 'treatment effect',
 'randomized clinical trial',
 'data collection instrument',
 'pooled AOR',
 'association',
 'power',
 "cohen's kappa",
 'pseudo-randomized',
 'treatment arm',
 'search string',
 'quasi-randomized',
 'cohort',
 'risk factors',
 'difference between means',
 'registry',
 'inter-rater reliability',
 'Odds Ratio',
 'placebo',
 'databases searched',
 'risk factor analysis',
 'difference in means',
 'random sample',
 'etiology',
 'i2']

# if generic_keyword not in study_designs, then add a design class
study_designs_value_list = [value for design in study_designs for value in study_designs[design]]
for a in generic_keywords:
    if a not in study_designs_value_list:
        study_designs[a] = [a]
        
len([value for design in study_designs for value in study_designs[design]])

In [ ]:
# def tag_study_design(study_designs):
#     df['study_design'] = [set() for _ in range(len(df))]
#     for tag in study_designs.keys():
#         for synonym in study_designs[tag]:
#             df[df.abstract.str.contains(synonym, case=False, na=False)].study_design.apply(lambda x: x.add(tag))

In [ ]:
def tag_study_design(study_designs):
    df['study_abstract'] = [set() for _ in range(len(df))]
    df['study_methods'] = [set() for _ in range(len(df))]
    df['study_results'] = [set() for _ in range(len(df))]

    for tag in study_designs.keys():
        for synonym in study_designs[tag]:
            df[df.abstract.str.contains(synonym, case=False, na=False) | df.title.str.contains(synonym, case=False, na=False)].study_abstract.apply(lambda x: x.add(tag))
            df[df.methods.str.contains(synonym, case=False, na=False)].study_methods.apply(lambda x: x.add(tag))
            df[df.results.str.contains(synonym, case=False, na=False)].study_results.apply(lambda x: x.add(tag))
    
    df['study_design'] = df.apply(lambda x: list(x.study_abstract.union(x.study_methods).union(x.study_results)), axis=1)
    df.study_abstract = df.study_abstract.apply(lambda x: list(x))
    df.study_methods = df.study_abstract.apply(lambda x: list(x))
    df.study_results = df.study_results.apply(lambda x: list(x))

tag_study_design(study_designs)

In [ ]:
df[df.study_design.str.len() != 0].tail(20).study_design

In [ ]:
len(df.study_abstract[df.study_abstract.str.len() != 0])

In [ ]:
len(df.study_methods[df.study_methods.str.len() != 0])

In [ ]:
len(df.study_results[df.study_results.str.len() != 0])

In [ ]:
len(df.study_design[df.study_design.str.len() != 0])

In [ ]:
len(df.study_design[(df.study_design.str.len() != 0) & df.is_covid19])

In [ ]:
df.drop(columns=['cord_uid', 'pmcid', 'pubmed_id', 'full_text_file', 'license', 'text_language',
                 'study_abstract', 'study_methods', 'study_results'], inplace=True)

# Export as .csv

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
save = 0

if save:
    df.to_hdf('meta/1-1-create_dataframe.h5', key='data')